In [ ]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload
from tqdm import tqdm

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)

In [ ]:

model = HookedTransformer.from_pretrained(
    "gpt2-small",
    # "pythia-2.8b",
    # "pythia-70m-deduped",
    # "tiny-stories-2L-33M",
    # "attn-only-2l",
    # center_unembed=True,
    # center_writing_weights=True,
    # fold_ln=True,
    # refactor_factored_attn_matrices=True,
    fold_ln=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)


In [ ]:


# path = "../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/1100001280_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152.pt"
path = "../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152:v9/final_sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
model(text, return_type="loss")

In [ ]:
from sae_training.utils import LMSparseAutoencoderSessionloader
model, sparse_autoencoder, activation_store = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)

In [ ]:
all_tokens_list = []
pbar = tqdm(range(128*6*4))
for i in pbar:
    batch_tokens = activation_store.get_batch_tokens()
    # get a random quart of the batch
    batch_tokens = batch_tokens[torch.randperm(batch_tokens.shape[0])][:batch_tokens.shape[0]//4]
    all_tokens_list.append(batch_tokens)
all_tokens = torch.cat(all_tokens_list, dim=0)
print(all_tokens.shape)
all_tokens = all_tokens[torch.randperm(all_tokens.shape[0])]
tokens = all_tokens[:4096*6]
del all_tokens
torch.mps.empty_cache()

In [ ]:
# path = "../week_8_jan/gpt2_small_features"
path = "../week_8_jan/gpt2_small_features_layer_5"

import os 
for i in range(sparse_autoencoder.cfg.d_sae):
    # print(f"Checking {i}")
    if not os.path.exists(f"{path}/data_{i:04}.html"):
        break 
    
n_features_at_a_time = 1024
id_of_last_feature_without_dashboard = i
n_features_remaining = sparse_autoencoder.cfg.d_sae - id_of_last_feature_without_dashboard
n_batches_to_do = n_features_remaining // n_features_at_a_time
id_to_start_from = sparse_autoencoder.cfg.d_sae - (n_batches_to_do + 1) * n_features_at_a_time
print(f"File {i} does not exist")
print(f"features left to do: {n_features_remaining}")
print(f"id_to_start_from: {id_to_start_from}")
print(f"number of batches to do: {(sparse_autoencoder.cfg.d_sae - id_to_start_from) // n_features_at_a_time}")

In [ ]:
from sae_analysis.visualizer import data_fns, model_fns, html_fns
import importlib

importlib.reload(data_fns)
importlib.reload(html_fns)
from sae_analysis.visualizer.data_fns import get_feature_data, FeatureData

# Currently, don't think much more time can be squeezed out of it. Maybe the best saving would be to
# make the entire sequence indexing parallelized, but that's possibly not worth it right now.


feature_idx = torch.tensor(range(id_to_start_from, sparse_autoencoder.cfg.d_sae)).reshape(-1, n_features_at_a_time)
feature_idx = [x.tolist() for x in feature_idx]
print(len(feature_idx))

In [ ]:
max_batch_size = 512

print(f"{sparse_autoencoder.cfg.hook_point_layer} {sparse_autoencoder.cfg.hook_point}")
print(f"{path}")

with torch.no_grad():
    for interesting_features in tqdm(feature_idx):
        print(interesting_features)
        feature_data = get_feature_data(
            encoder=sparse_autoencoder,
            # encoder_B=sparse_autoencoder,
            model=model,
            hook_point=sparse_autoencoder.cfg.hook_point,
            hook_point_layer=sparse_autoencoder.cfg.hook_point_layer,
            hook_point_head_index=None,
            tokens=tokens,
            feature_idx=interesting_features,
            max_batch_size=max_batch_size,
            left_hand_k = 3,
            buffer = (5, 5),
            n_groups = 10,
            first_group_size = 20,
            other_groups_size = 5,
            verbose = True,
        )
        
        for test_idx in feature_data.keys():
            html_str = feature_data[test_idx].get_all_html()
            with open(f"{path}/data_{test_idx:04}.html", "w") as f:
                f.write(html_str)


